In [215]:
import numpy as np
import pandas as pd
import random
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [189]:
#the function below takes a state as a string and return it's correspondant index to be used for
# Q and R matrix
def find(state):
    return list_of_states.index(state)
#this function productes a pandas dataframe that has indexes as states and columns as 
#actions. this is useful to visualise the Q and R matrix.
def clean_RDF(reward_going_to_action):
    R_matrix=np.array([[np.nan for a in all_routes.keys()] for s in all_routes.keys()])
#     for i in range(0,41):
#         R_matrix[i,i]=reward_going_to_self
    R_DF=pd.DataFrame(data=R_matrix,index=all_routes.keys(),columns=all_routes.keys())
    for start in all_routes.keys():
        list_of_connections=all_routes[start]
        R_DF.loc[start][list_of_connections]=reward_going_to_action
    return R_DF
def make_DF(matrix):
    return pd.DataFrame(data=matrix,index=all_routes.keys(),columns=all_routes.keys())
#this function initializes an empty dataframe, an R Matrix and a Q Matrix
def create_environment(reward_action=-1):
    R_DF=clean_RDF(reward_action)
    R_matrix=R_DF.values
    Q_matrix=np.zeros(R_matrix.shape)
    return R_DF,R_matrix,Q_matrix

#main learning algorithm based on TD learnign and Q learning
def train_AGENT(epsilon=0.99
                ,epsilon_decay_rate=0.99
                ,alpha=1
                ,gamma=0.8
                ,number_episodes=200
                ,number_of_movements_per_episode=200
                ,destination_type='random'
                ,reward_going_to_action=-1
                ,reward_going_to_destination=100):
#     reward_going_to_self=-10
    R_DF,R_matrix,Q_matrix=create_environment(reward_going_to_action)
    rewards_measurements=[]
    #it's a way to monitor convergence of the Q matrix 
    changes_per_episode=[] 
    episodes=[]
    movements_per_episode=[]
    #hyperparameters

    list_of_states=list(all_routes.keys())
    if destination_type=='random':
        destination=random.choice(list_of_states)
    else:
        destination=destination_type
    R_DF.loc[R_DF[destination]==reward_going_to_action,destination]=reward_going_to_destination
    
    print('going to :{}'.format(destination))
    R_matrix=R_DF.values
    for episode in range(number_episodes):

        state=random.choice(list_of_states)
        while state==destination:
            state=random.choice(list_of_states)
        initial_state=state
        state_ind=find(state)

    #     if state==destination:
    #         state=random.choice(list_of_states)
    #     print('going from {} to : {}'.format(state,destination))
        # state="king's cross"
        # initial_state=state\\\
        # state_ind=find(state)
        # destination="south kensington"
        #Updating the R Matrix with the reward goal of the destination
    #     R_DF=clean_RDF()
    #     R_DF.loc[R_DF[destination]==reward_going_to_action,destination]=reward_going_to_destination
    #     R_matrix=R_DF.values

        rewards_per_episode=0
        changes=0
        #starting the episodes loop
        for movement in range(number_of_movements_per_episode):
            possible_actions_ind=np.where(~np.isnan(R_matrix[state_ind]))[0]
            possible_actions=[]
            for act_pos in possible_actions_ind:
                possible_actions.append(list_of_states[act_pos])
    #         print('all possible actions from {} are :'.format(list_of_states[state_ind]))
    #         print(possible_actions)

            possible_actions=np.array(possible_actions)

            Q_values_of_possible_actions=Q_matrix[state_ind,possible_actions_ind]

    #         print('the possible actions have these Q values '.format(Q_values_of_possible_actions))
    #         print(Q_values_of_possible_actions)

            best_actions=possible_actions_ind[np.where(Q_values_of_possible_actions==
                                                   np.max(Q_values_of_possible_actions))]

            best_actions_Q=[Q_matrix[state_ind,x] for x in best_actions]

    #         if len(best_actions)>1:
    #             print('multiple actions from state {} have the maximum q value. we will select a random action among these'.format(state))
    #             print('best available actions are {} with a value of {}'.format(
    #                 [list_of_states[x] for x in best_actions],best_actions_Q))


            if np.random.uniform() > 1-epsilon:
                a=np.random.choice(possible_actions_ind)
    #             print('Selecting random action :{} with current Q :{}'.format(list_of_states[a],Q_matrix[state_ind,a]))
            else:
                a=np.random.choice(best_actions)
    #             print('Selecting Greedy action :{} with current Q :{}'.format(list_of_states[a],Q_matrix[state_ind,a]))


            reward=R_matrix[state_ind,a]
            rewards_per_episode+=reward
    #         print("Reward for taking action '{}' from state '{}': {}".format(list_of_states[a],
    #                                                                          list_of_states[state_ind],
    #                                                                          reward))


            old_state=state_ind

            state_ind=a

    #         print('after taking an action :{} from state :{} the new state is :{}'.format(
    #         list_of_states[a],list_of_states[old_state],list_of_states[state_ind]))

            q_TD=Q_matrix[old_state,a] + alpha * (reward + gamma * np.max(Q_matrix[state_ind,:]) -
                                                 Q_matrix[old_state,a])

            old_Q = Q_matrix[old_state,a]
            Q_matrix[old_state,a]=q_TD


            update_value=q_TD - old_Q 

            changes+=update_value





    #         print('Q value updates' \
    #               "Q( {} , {} ) = Q( {} , {} ) + alpha * ( r ( {} , {} ) + gamma * max( Q [ {} ,:]) - Q[ {} ,  {} ])".format(
    #         list_of_states[old_state],list_of_states[a],list_of_states[old_state],
    #         list_of_states[a],list_of_states[old_state],list_of_states[a],list_of_states[state_ind],
    #         list_of_states[old_state],list_of_states[a]))


    #         print("Q matrix update: " \
    #               "Q( {} , {} ) = {} + {} * ( {} + {} * {} - {} ) = {}".format(
    #             list_of_states[old_state], list_of_states[a], Q_matrix[old_state,a].round(0),
    #             alpha, reward, gamma, np.max(Q_matrix[state_ind,:]).round(0), 
    #             Q_matrix[old_state,a].round(0), q_TD))


    #         print('Q matrix updated: \n\n {}'.format(Q_matrix.round(0)))

            if list_of_states[state_ind] == destination:
                print("Goal state '{}' reached. Ending episode.".format(destination))
                print('The destination was reached after {} steps'.format(movement))
                print('the cumulative rewards for the episode ( {} to : {} ) is :{} '.format(initial_state,destination,
                                                                                             rewards_per_episode))
                break
        epsilon*=epsilon_decay_rate
        
        rewards_measurements.append(rewards_per_episode)
        changes_per_episode.append(changes)
        episodes.append(episode)
        movements_per_episode.append(movement)
        
    return rewards_measurements,changes_per_episode,episodes,movements_per_episode,Q_matrix


initiating the environment constants

In [190]:
#these are all the states (stations) of our chosen environment. if we want to expand to other 
#zones of the london undeground we can expand on this list
all_stations=['Paddington', 'edgware road', 'marylebone', 'baker street', "regent's park", 'oxford circus',
              'piccadilly circus', 'charing cross', 'embankment', 'waterloo', 'elephant and castle',
             'Notting hill', 'queensway', 'lancaster gate', 'marble arch', 'bond street', 
              'oxford circus', 'tottenham court road', 'holborn','chancery lane', 'St.pauls','bank',
              'liverpool street','Euston', 'Warren street', 'goodge street', 'Tottenham Court road', 
              'Leicester Square', 'charing cross','embankment', 'waterloo','baker street', 'bond street'
              , 'green park', 'westminster', 'waterloo', 'southwark', 'london bridge','vauxhall', 
              'pimlico', 'Victoria', 'green park', 'oxford circus', 'warren street', "Euston","king's cross",
             "Earl's Court", 'Gloucester Road', 'South Kensington', 'Knightsbridge', 'hyde park corner', 
              'green park', 'piccadilly circus','Leicester square', 'covent garden', 'holborn', 'Russel square', 
              "king's cross"]

In [191]:
#these are the connections from states to other states. the Dictionary represents the possible
#actions from state Key.
all_routes={'paddington':'edgware road',
           'edgware road':['marylebone','paddington'],
           'marylebone':['edgware road','baker street'],
           'baker street': ['marylebone',"regent's park","bond street"],
            "regent's park":['oxford circus','baker street'],
            'piccadilly circus':['oxford circus','charing cross','green park','leicester square'],
            'charing cross':['embankment','leicester square','piccadilly circus'],
            'embankment':['waterloo','charing cross'],
            'waterloo':['elephant and castle','southwark' ,'embankment','westminster'],
            "earl's court":'gloucester road',
            'gloucester road':['south kensington',"earl's court"],
            'south kensington':['knightsbridge','gloucester road'],
            'knightsbridge':['hyde park corner','south kensington'],
            'hyde park corner':['green park','knightsbridge'],
            'green park':['westminster','victoria','bond street','hyde park corner','piccadilly circus',
                         'oxford circus'],
            'leicester square':['covent garden','piccadilly circus','charing cross',
                                'tottenham court road'],
            'covent garden':['leicester square','holborn'],
            'holborn':['covent garden','tottenham court road','chancery lane','russel square'],
            'notting hill':'queensway',
            'queensway':['leicester square','notting hill'],
            'lancaster gate':['marble arch','queensway'],
            'marble arch':['bond street','lancaster gate'],
            'bond street':['oxford circus','marble arch','baker street','green park'],
            'oxford circus':['bond street',"regent's park",'tottenham court road','piccadilly circus',
                            'green park','warren street'],
            'tottenham court road':['oxford circus','holborn','leicester square','goodge street'],
            'holborn':['chancery lane','tottenham court road'],
            'chancery lane':['holborn','st.pauls'],
            'st.pauls':['bank','chancery lane'],
            'bank':['st.pauls','liverpool street'],
            'liverpool street':'bank',
            'vauxhall':'pimlico',
            'pimlico':['victoria','vauxhall'],
            'victoria':['green park','pimlico'],
            'green park':['victoria','oxford circus','piccadilly circus','hyde park corner',
                         'bond street','westminster'],
           
            'warren street':['oxford circus','euston','goodge street'],
            'euston':['warren street',"king's cross"],
            'goodge street':['warren street','tottenham court road'],
            'westminster':['green park','waterloo'],
            'southwark':['waterloo','london bridge'],
           'london bridge':'southwark',
           'russel square':'holborn',
           "king's cross":['euston','russel square'],
           'elephant and castle':'waterloo'}

